In [ ]:
from __future__ import unicode_literals
import os
from gensim import corpora, models
import string
import nltk
import numpy as np
from langdetect import detect
import re
import pickle
from pattern.text.fr import parse
import math


In [ ]:
# Fonction qui permet de transformer les mots d'un texte en lemmes
def lemmatization(content_file):
    content_file = parse(content_file, relations=True, lemmata=True).split(" ")
    res = [elt.split("/")[5] for elt in content_file]
    return " ".join(res)



# Construction d'un stemmer pour le français
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()



# Recuperation de la liste des stopwords depuis un fichier 
import codecs
file_stopwords = codecs.open("frenchST.txt","r", encoding="utf-8")
stopwords = file_stopwords.read().split("\n")
file_stopwords.close()


# Creation d'un set de symboles de ponctuation qui servira a la suppression de la ponctuation
ponctuation = set(string.punctuation)


# Fonction qui permet de supprimer les mots peu frequents d'un document
# La notion de "peu frequent" est defini par le parametre "min_freq"
# Tout mot dont la frequence dans le document est < à min_freq est supprime
def remove_infrequent_words(content_file, min_freq):
    content_file = np.array(content_file.split())
    # creation d'une distribution de frequences sur le contenu du fichier en utilisant nltk
    freq_dist = nltk.FreqDist(content_file) 
    # On obtient un dictionnaire des mots du fichier avec leur frequence
    for elt in freq_dist:  # elt represente une cle, c'est le mot 
        if freq_dist[elt] < min_freq: #freq_dist[elt] est la frequence du mot elt
            content_file = np.delete(content_file, np.where(content_file == elt)[0])
    return ' '.join(content_file)


# Fonction qui permet de raciniser les mots d'un texte
def stemming(content_file):
    content_file = content_file.split()
    for word in range(len(content_file)):
        content_file[word] = stemmer.stem(content_file[word].decode("utf-8"))
    return ' '.join(content_file)



def preprocess_data_file(content_file):
    content_file = content_file.lower()  # Faire passer les caracteres du texte du majuscule au minuscule 
    # Supprimer quelques caracteres speciaux non traites avec le set de ponctuation
    content_file = content_file.replace('’', "")
    content_file = content_file.replace('«', "")
    content_file = content_file.replace('»', "")
    content_file = content_file.replace('—', "")
    content_file = content_file.replace('\'', "")
    content_file = content_file.replace('©', "")
    content_file = content_file.replace('–', "")
    content_file = content_file.replace('¿', )
    # Supprimer les nombres
    content_file = re.sub("[0-9]+", "", content_file)
    # Supprimer la ponctuation 
    content_file = ''.join(char for char in content_file if char not in ponctuation) 
    # Appel de la fonction qui supprime les mots peu frequents, ici avec frequence minimale de 2
    content_file = remove_infrequent_words(content_file, 2)
    #content_file = stemming(content_file)
    return content_file



#_____________________________________Preparation du corpus_________________________________________________
# Initialisation du chemin sur le repertoir contenant le corpus
chemin_corpus = os.getcwd()+"/Subset/"
# Recuperer la liste des sous dossiers
list_dir = os.listdir(chemin_corpus)
infopath = [] # Structure pour stocker les chemins des fichiers (necessaire pour la visualisation)
infolength=[] # Structure pour stocker les longueurs des fichiers (necessaire pour la visualisation)
texts  =[] # liste des documents apres pre-traitement

for dir_annee in list_dir:
    sub_dirs_annee = os.listdir(chemin_corpus+dir_annee)   #recuperer les sous dossiers correspondants aux annees
    for sub_dir in sub_dirs_annee:
        sub_sub_dirs = os.listdir(chemin_corpus+dir_annee+"/"+sub_dir) #recuperer les sous dossiers correspondants aux mois
        for sub_sub_dir in sub_sub_dirs:
            files = os.listdir(chemin_corpus+dir_annee+"/"+sub_dir+"/"+sub_sub_dir) # recupere les sous dossiers correspondants aux jours
            for f in files: # pour chaque fichier...
                if os.stat(chemin_corpus+dir_annee+"/"+sub_dir+"/"+sub_sub_dir+"/"+f).st_size!=0: # Verifier que le fichier n'est pas vide
                    open_file = codecs.open(chemin_corpus+dir_annee+"/"+sub_dir+"/"+sub_sub_dir+"/"+f, "r", encoding="utf-8")
                    content_file = open_file.read()
                    if detect(content_file)== "fr": # Confirmer que la langue c'est bien du français
                        content_file = lemmatization(content_file) # Appliquer la lemmatization
                        content_file = preprocess_data_file(content_file) # Appliquer d'autres pre-traitements
                        content_file2 = []
                        # Supprimer les stop words
                        # La boucle a ete formulee ainsi et non en (if word in ...) a cause de problemes d'encodage 
                        for word in content_file.lower().split():
                            existe = False
                            for stopword in stopwords:
                                if word == stopword:
                                    existe = True
                                    break
                            if existe != True:
                                content_file2.append(word)

                        content_file = content_file2 # contenu du fichier pre-traité

                        # Eliminer les fichiers qui après pre-traitement deviennent vides ou trop petits
                        if len(content_file) > 5:
                            texts.append(content_file)
                            # Ajouter le chemin et la longueur du fichier dans les structures precedentes
                            infopath.append(chemin_corpus+dir_annee+"/"+sub_dir+"/"+sub_sub_dir+"/"+f)
                            infolength.append(len(content_file))
                    open_file.close()
#___________________________________________________________________________________________________________



In [ ]:
# Supprimer les accents qui posent problemes en normalisant a l'unicode
import unicodedata
texts = [[unicodedata.normalize('NFD', word).encode('ascii', 'ignore') for word in text] for text in texts]

In [ ]:
# Enregistrer texts, infopath et infolength pour ne pas reexecuter le code a chaque fois
pickle.dump(texts, open(os.getcwd()+"/10000/texts.obj", "wb"))
pickle.dump(infopath, open(os.getcwd()+"/10000/infopath.obj", "wb"))
pickle.dump(infolength, open(os.getcwd()+"/10000/infolength.obj", "wb"))

In [ ]:
# Preparation des INPUT du LDA
# creation d'un dictionnaire qui attribu a chaque mot de la collection un identifiant unique
dictionary = corpora.Dictionary(texts) 

# mapper le corpus de la structure basique a une structure de liste de document 
# chaque document est une liste de tuples (mot, nombre d'occurence dans le document)
corpus = [dictionary.doc2bow(text) for text in texts] #term_frequency

# Appel de lda Multicore plutot que le lda basique
# lda Multicore permet une execution plus rapide en specifiant le nombre de workers 
# chunksize : lenombre de documents qui peuvent etre charges en memoire
# num_topics: nombre de topics
# passes : nombre de passages d'apprentissage sur le corpus que l'on veut effectuer
lda = models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=50, workers = 2,chunksize=10000, passes=1)

#lda.update(other_corpus) # Fonction qui permet d'enrichir le lda avec d'autres donnees

# Visualisation des top topics si necessaire
#top = lda.top_topics(corpus=corpus)
#for top_topic in top:
#    print top_topic


# Enregistrer le output LDA
#pickle.dump(lda, open(os.getcwd()+"/lda_output", "wb"))


# Recharger le output LDA
#lda = pickle.load(open(os.getcwd()+"/lda_output", "rb"))

In [ ]:
#______________Preparation des input pour la visualisation______________________
# Distribution des probabilites qu'un document i (lignes) ait un topic j (colonnes)
def doc_topic_dists(corpus, lda):
    doc_topic_dists = []
    for doc in range(len(corpus)):
        list_topic_proba = []
        temp = lda.get_document_topics(corpus[doc], minimum_probability=0)
        for topic, proba in temp:
            list_topic_proba.append(proba)
        doc_topic_dists.append(list_topic_proba)
    return doc_topic_dists
#doc_topic_dists = doc_topic_dists(corpus, lda) 

# Distribution des probabilites qu'un topic i (lignes) contienne un term j (colonnes)
def topic_term_dists(lda, num_topics, len_vocab):
    topic_term_dists = []
    for topic in range(num_topics):
        list_term_proba = [0]*len_vocab
        temp = lda.get_topic_terms(topic)
        for term, proba in temp:
            list_term_proba[term] = proba
        topic_term_dists.append(list_term_proba)
    return topic_term_dists
# topic_term_dists = topic_term_dists(lda, 5, len(dictionary.token2id))


#Pour avoir tout le vocabulaire ou chaque mot est represente par son identifiant
def get_vocabularyIDs(dictionary_tokens):
    vocabulary = []
    for token in dictionary_tokens:
        vocabulary.append(dictionary_tokens[token])
    return vocabulary
# vocabularyIDs = get_vocabularyIDs(dictionary.token2id)  


#Pour avoir tout le vocabulaire ou chaque mot est represente dans sa forme normale
def get_vocabularyAlpha(dictionary_tokens):
    vocabulary = []
    for token in dictionary_tokens:
        vocabulary.append(dictionary_tokens[token])
    return vocabulary
# vocabulary_alpha = get_vocabularyAlpha(dictionary.id2token) 


# Recuperation de la longueur des fichiers
def doc_lengths(texts):
    doc_lengths = []
    for text in texts:
        doc_lengths.append(len(text))
    return doc_lengths
#____________________________________________________________________________________
